In [2]:
!pip install confluent_kafka

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 66.9 MB/s eta 0:00:00


In [20]:
import random
from confluent_kafka import Producer
from confluent_kafka import Consumer
import time, json

In [14]:
import logging

# Configure logging for the 'confluent_kafka' logger
logging.basicConfig(level=logging.DEBUG)  # Set the desired log level (DEBUG, INFO, WARNING, ERROR, CRITICAL)
logger = logging.getLogger('confluent_kafka')


In [15]:
config = {
        # User-specific properties that you must set
        'bootstrap.servers': 'pkc-xrnwx.asia-south2.gcp.confluent.cloud:9092',
        'sasl.username':     'L5TEVAYKNYRO3VVF',
        'sasl.password':     'uSyvdswvZgWyZoNPycDwCRsklSZmJ4qZG3FrgAAKqwhNEV/pq0O15o4jfTR5S9ye',

        # Fixed properties
        'security.protocol': 'SASL_SSL',
        'sasl.mechanisms':   'PLAIN',
        'debug': 'all',
        'acks':              'all'
    }

    # Create Producer instance
producer = Producer(config)


In [21]:

# Define lists of words with positive and negative connotations
positive_words = ["good", "great", "excellent", "delicious", "amazing", "wonderful", "fantastic", "enjoyable"]
negative_words = ["bad", "terrible", "awful", "horrible", "disgusting", "unpleasant", "disappointing"]

objects = ["food", "taste", "ambience", "service", "experience"]

def generate_review_with_rating():
    """Generates a review with an appropriate rating based on sentiment."""
    # Randomly choose positive or negative sentiment
    sentiment = random.choice(["positive", "negative"])

    # Select words based on sentiment
    if sentiment == "positive":
        words = random.sample(positive_words, random.randint(1, 3))  # Choose 1-3 positive words
        rating = random.randint(4, 5)  # Assign a rating of 4 or 5
    else:
        words = random.sample(negative_words, random.randint(1, 3))  # Choose 1-3 negative words
        rating = random.randint(1, 3)  # Assign a rating of 1, 2, or 3

    # Construct the review
    review = " ".join(words) + " " + random.choice(objects)

    return review, rating

In [22]:
import json

# ... other imports and configurations ...
topic = "reviews"
def delivery_callback(err, msg):
  if err:
    print('ERROR: Message failed delivery: {}'.format(err))
  else:
    print("Produced event to topic {topic}: key = {key:12} value = {value:12}".format(
      topic=msg.topic(),
      key=msg.key().decode('utf-8'),
      value=msg.value().decode('utf-8')))


count = 0
for _ in range(10):
  review, rating = generate_review_with_rating()
  # Create a dictionary containing the review and rating
  data = {'review': review, 'rating': rating}
  # Convert the dictionary to a JSON string
  producer.produce(topic, key=str(count), value=json.dumps(data).encode('utf-8'), callback=delivery_callback) #changed here
  count += 1
  # Block until the messages are sent.
producer.poll(10000)
producer.flush()

Produced event to topic reviews: key = 7            value = {"review": "disappointing horrible service", "rating": 2}
Produced event to topic reviews: key = 4            value = {"review": "good ambience", "rating": 4}
Produced event to topic reviews: key = 5            value = {"review": "excellent food", "rating": 4}
Produced event to topic reviews: key = 6            value = {"review": "wonderful fantastic experience", "rating": 5}
Produced event to topic reviews: key = 9            value = {"review": "delicious excellent ambience", "rating": 5}
Produced event to topic reviews: key = 0            value = {"review": "bad disgusting ambience", "rating": 3}
Produced event to topic reviews: key = 1            value = {"review": "enjoyable service", "rating": 4}
Produced event to topic reviews: key = 8            value = {"review": "disgusting disappointing service", "rating": 1}
Produced event to topic reviews: key = 2            value = {"review": "wonderful excellent great taste", "ra

0

In [17]:
logging.basicConfig(filename='producer.log', level=logging.DEBUG)

In [18]:
# Create a custom log handler to redirect logs to the cell output
class IPythonLogHandler(logging.Handler):
    def emit(self, record):
        get_ipython().log.info(self.format(record))

# Add the custom handler to the logger
logger.addHandler(IPythonLogHandler())